In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [61.8 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-12 22:50:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.70MB/s    in 0.2s    

2021-06-12 22:50:07 (5.70 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

## Deliverable 1
### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date,count
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg(count('customer_id').alias("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   26079415|             1|
|   12521601|             2|
|    4593696|             1|
|    1468259|             1|
|   38173350|             1|
|    3953141|             2|
|   20598424|             1|
|   46625306|             1|
|   47160828|             2|
|    1337034|             1|
|   14231016|             1|
|   49785748|             1|
|     712525|             1|
|   35339694|             1|
|    8211293|             1|
|   41832251|             1|
|    9037272|             1|
|   52036002|             1|
|   49514578|             1|
|   22875949|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select('product_id','product_title').dropDuplicates(['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001KC02UA|Happy Hour! [Down...|
|B002FGU06Q|Tradewinds Odysse...|
|B002MUB0N2|Word Harmony [Dow...|
|B002N5NAXY|Continental Cafe ...|
|B0030EFVA2|688I Hunter Kille...|
|B0030GG0XC|Super Jigsaw Pupp...|
|B0031561G8|Crazy Machines: N...|
|B0031RGGOI|Next Life [Download]|
|B0037HOOTQ|Death Track: Resu...|
|B003O68T1Y|Casino Island To ...|
|B003OA4CSY|Tecmo Bowl: Throw...|
|B0042JTQWS|Jewel Drop! [Down...|
|B004MMEGHU|Matches & Matrimo...|
|B0054KC23E|Pizza Deliciozo [...|
|B0056JGF9U|Pirate Mysteries:...|
|B0058JZ0WQ|F1 Chequered Flag...|
|B005OE9YUY|        Nuclear Dawn|
|B005Q1YA5O|    The Sims 3: Pets|
|B005R40HFW|Sinbad: In search...|
|B008BKF3OE|RuneScape [Game C...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193| 2015-08-31|
|R1WFOQ3N9BO65I|     133437|B00F4CEHNK|     341969535| 2015-08-31|
| R3YOOS71KM5M9|   45765011|B00DNHLFQA|     951665344| 2015-08-31|
|R3R14UATT3OUFU|     113118|B004RMK5QG|     395682204| 2015-08-31|
| RV2W9SGDNQA2C|   22151364|B00G9BNLQE|     640460561| 2015-08-31|
|R3CFKLIZ0I2KOB|   22151364|B00IMIL498|     621922192| 2015-08-31|
|R1LRYU1V0T3O38|   38426028|B00S00IJH8|     215163395| 2015-08-31|
| R44QKV6FE5CJ2|    6057518|B004RMK4BC|     384246568| 2015-08-31|
|R2TX1KLPXXXNYS|   20715661|B00K59HKIQ|     384246568| 2015-08-31|
|R1JEEW4C6R89BA|   26540306|B00K59HKIQ|     384246568| 2015-08-31|
|R3B3UHK1FO0ERS|    8926809|B004774IPU|     151985175| 2015-08-31|
|R2GVSDHW513SS1|   31525534|B002LIT9EC|     695277014| 2015-08

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select('review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase')
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|                Y|
|R1LRYU1V0T3O38|          4|            0|          0|   N|                Y|
| R44QKV6FE5CJ2|          5|            0|          0|   N|                Y|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|                Y|
|R1JEEW4C6R89BA|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cnyqg36t2sgt.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "123456789", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

## Deliverable 2
### Filter the data and create a new DataFrame

In [ ]:
new_df = df.filter((df['total_votes']>=20))
new_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12569493| RV4UH0OWW97K4|B0149HT55K|     947161997|Megaman Legacy Co...|Digital_Video_Games|          5|           19|         22|   N|                N|Mega Man Legacy C...|What is Mega Man?...| 2015-08-29|
|         US|   40702997| RG6GT6G9WNYT1|B013XJ2M8M|     628348017|Rocket League [On...|Digital_Video_Games| 

### helpful_votes/total_votes >= 50%

In [ ]:
helpful_df = new_df.filter(new_df['helpful_votes']/new_df['total_votes']>=0.5)
helpful_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12569493| RV4UH0OWW97K4|B0149HT55K|     947161997|Megaman Legacy Co...|Digital_Video_Games|          5|           19|         22|   N|                N|Mega Man Legacy C...|What is Mega Man?...| 2015-08-29|
|         US|   40702997| RG6GT6G9WNYT1|B013XJ2M8M|     628348017|Rocket League [On...|Digital_Video_Games| 

### Vine program reviews

In [ ]:
vine_df = helpful_df.filter(helpful_df['vine']=='Y')
vine_df.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [ ]:
nonvine_df = helpful_df.filter(helpful_df['vine']=='N')
nonvine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12569493| RV4UH0OWW97K4|B0149HT55K|     947161997|Megaman Legacy Co...|Digital_Video_Games|          5|           19|         22|   N|                N|Mega Man Legacy C...|What is Mega Man?...| 2015-08-29|
|         US|   40702997| RG6GT6G9WNYT1|B013XJ2M8M|     628348017|Rocket League [On...|Digital_Video_Games| 

### Determine total number of reviews, 5-star reviews and percentage.

In [ ]:
# number of vine reviews
num_reviews_vine = vine_df.count()
print('Total number of vine reviews is ' + str(num_reviews_vine))
# number of 5-star vine reviews
num_reviews_vine_5_star = vine_df.filter(vine_df['star_rating']==5).count()
print('Total number of vine reviews with 5-star rating is ' + str(num_reviews_vine_5_star))
# percentage of 5-star reviews
if num_reviews_vine == 0:
  print('5-start precenage is N/A')
else:
  vine_5_star_pct = num_reviews_vine_5_star/num_reviews_vine*100
  print('5-start precenage is ' + str(vine_5_star_pct) + '%')

Total number of vine reviews is 0
Total number of vine reviews with 5-star rating is 0
5-start precenage is N/A


In [ ]:
# number of vine reviews
num_reviews_nonvine = nonvine_df.count()
print('Total number of non-vine reviews is ' + str(num_reviews_nonvine))
# number of 5-star vine reviews
num_reviews_nonvine_5_star = nonvine_df.filter(nonvine_df['star_rating']==5).count()
print('Total number of non-vine reviews with 5-star rating is ' + str(num_reviews_nonvine_5_star))
# percentage of 5-star reviews
if num_reviews_nonvine == 0:
  print('5-start precenage is N/A')
else:
  nonvine_5_star_pct = num_reviews_nonvine_5_star/num_reviews_nonvine*100
  print('5-start precenage is ' + str(round(nonvine_5_star_pct,2))+ '%')

Total number of non-vine reviews is 1685
Total number of non-vine reviews with 5-star rating is 631
5-start precenage is 37.45%
